<a href="https://colab.research.google.com/github/reulke/POL/blob/main/231109_Pol01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install polanalyser

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Show the images with bounding boxes in Colab
import os
import numpy as np
from PIL import Image
import polanalyser as pa
from matplotlib import pyplot as plt
# from IPython.display import display
import cv2
from pathlib import Path


visualization_dir = "/content/drive/MyDrive/img/pol/240905/"

for viz_file_name in os.listdir(visualization_dir):
  print(viz_file_name)
  file_name = os.path.basename(viz_file_name)
  print('name ',file_name)

  img = Image.open(os.path.join(visualization_dir, viz_file_name))
  im = np.array(img, dtype=np.float64)

  # Display basic information about the image
  # print(im.shape,im.dtype)

  # Demosaicing
  img_000, img_045, img_090, img_135 = pa.demosaicing(im, pa.COLOR_PolarMono)
  # print(img_000.shape,img_000.dtype)

  # Calculate the Stokes vector per-pixel
  image_list = [img_000, img_045, img_090, img_135]
  angles = np.deg2rad([0, 45, 90, 135])
  img_stokes = pa.calcStokes(image_list, angles)
  # print(img_stokes.shape,img_stokes.dtype)

  # Decompose the Stokes vector into its components
  img_s0, img_s1, img_s2 = cv2.split(img_stokes)

  # Convert the Stokes vector to Intensity, DoLP and AoLP
  img_intensity = pa.cvtStokesToIntensity(img_stokes)
  img_dolp = pa.cvtStokesToDoLP(img_stokes)
  img_aolp = pa.cvtStokesToAoLP(img_stokes)
  # print(img_intensity.shape,img_intensity.dtype)

  oname = file_name + '_intensity.tiff'
  cv2.imwrite(os.path.join(visualization_dir, oname), img_intensity)
  oname = file_name + '_dolp.tiff'
  cv2.imwrite(os.path.join(visualization_dir, oname), img_dolp)
  oname = file_name + '_aolp.tiff'
  cv2.imwrite(os.path.join(visualization_dir, oname), img_aolp)


767_010.tiff
name  767_010.tiff
(2048, 2448) float64
(2048, 2448) float64
(2048, 2448, 3) float64
767_011.tiff
name  767_011.tiff
(2048, 2448) float64
(2048, 2448) float64
(2048, 2448, 3) float64
767_012.tiff
name  767_012.tiff
(2048, 2448) float64
(2048, 2448) float64
(2048, 2448, 3) float64


In [ ]:
import cv2
import polanalyser as pa

# Read all images
path = "/content/drive/MyDrive/img/pol/toy_example_3x3_pc"
pcontainer = pa.PolarizationContainer(path)
image_list = pcontainer.get_list("image")
mueller_psg_list = pcontainer.get_list("mueller_psg")
mueller_psa_list = pcontainer.get_list("mueller_psa")

print(len(pcontainer))  # 16
print(image_list[0].shape)  # (2048, 2448)
print(mueller_psg_list[0].shape)  # (3, 3)
print(mueller_psa_list[0].shape)  # (3, 3)

# Calculate Mueller matrix
img_mueller = pa.calcMueller(image_list, mueller_psg_list, mueller_psa_list)

print(img_mueller.shape)  # (2048, 2448, 3, 3)

# Visualize Mueller matrix image
pa.plotMueller("plot_mueller.png", img_mueller, vabsmax=2.0)

FileNotFoundError: ignored